# 3-Stage Clos Network with Static VXLAN to make Networks Scalable
###### <sup>Inputs: Bhavani Parise, Deepti Chandra; Developed by Sarah Samuel; Read the use case on Devnet: https://developer.cisco.com/docs/sonic/#!3-stage-clos-network-with-static-vxlan-to-make-networks-scalable</sup>


The [3 Stage Clos Notebook](./3StageClos-Notebook.ipynb) walked you through the step-by-step procedure to configure a 3 stage Clos network on Cisco 8000 routers that run SONiC operating system. This notebook shows you how to configure Static VXLAN over a 3-stage Clos network underlay.

VXLAN is a tunneling protocol that stretches Layer 2 networks over an underlying Layer 3 IP network by encapsulating Layer 2 Ethernet frames within Layer 4 User Datagram Protocol (UDP). It then transports the encapsulated frames over a Layer 3 network.

You can create up to 16 million VXLANs instead of the traditional VLAN, which allows only 4096 VLANs in a network. Thus, VXLAN enables the building of highly scalable networks with physically distant Layer 2 network segments.

There are 2 types of VXLANS:

1. **Ethernet virtual private network (EVPN) VXLANs** - EVPN provides the control plane functionality for these VXLANs.

2. **Static VXLANs** - The router does not maintain a control plane for static VXLANs. So you should manually configure virtual tunnel end points and routes.

Though you use generally switches in a 3-stage Clos network, this notebook demonstrates how to [Bring Up 3-Stage Clos Network as an Underlay Network](#bring-up-3-stage-clos-network-as-an-underlay-network) using Cisco 8000 series routers that run SONiC. You can then [Configure Static VXLANs over the 3-Stage Clos Network](#configure-static-vxlans-over-the-3-stage-clos-network).

The following topology diagram depicts a simple 3-stage Clos network with two leaf routers in Tier-0 and two spine routers in Tier-1.

<center><img src="./images/522559.jpg" width="700"/></center>


This topology shows the static VXLAN overlay on the 3-stage Clos network:

<center><img src="./images/522560.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the live network nodes on the Cisco 8000 Emulator that runs in the background. The notebook refreshes the output of each cell-execution just beneath it.

Play the code-cells in the sections [Access Device Consoles](#Access-Device-Consoles) and [Bring Up 3 Stage Clos Network](#Bring-Up-3-Stage-Clos-Network).

When the 3-stage Clos network is up, [Configure Static VXLANs over the 3-Stage Clos Network](#Configure-Static-VXLANs-over-the-3-Stage-Clos-Network), by playing through these steps:
* [Configure VLAN on Leaf Nodes](#Configure-VLAN-on-Leaf-Nodes)
* [Set Up and Apply Static VXLAN Configurations](#Set-Up-and-Apply-Static-VXLAN-Configurations)
* [Set Up and Load VNet Route Tables](#Set-Up-and-Load-VNet-Route-Tables)
* [Verify Static VXLAN](#Verify-Static-VXLAN)
* [Send Traffic across VXLAN Tunnel](#Send-Traffic-across-VXLAN-Tunnel)
* [Tested Static VXLAN Scale](#Tested-Static-VXLAN-Scale)

Finally, [clean up emulator session](#Clean-Up-Emulator-Session), once you are done.

## Access Device Consoles

> Play the following cell to bring up topology and access ssh console of each device in the topology. This cell takes about 10 - 15 minutes to complete execution. When you see the **Sim status** displayed as  **{'localhost': 'running'}** underneath this cell, the topology is up. Wait for a few more seconds until the code accesses the ssh console of each device. 

> Avoid playing this cell more than once, without [cleaning up the emulator session](#Clean-Up-Emulator-Session).

In [ ]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }
        
tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

## Bring Up 3-Stage Clos Network

> To configure a 3-stage clos network, play through the following code-cells:

In [ ]:
# Configure Host-Names
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')

# Assign IP Addresses
print ("\n******************** Assign IP addresses on S0 *************************")
out = nodes['S0'].execute('sudo config interface ip add Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
print ("\n******************** Assign IP addresses on S1 *************************")
out = nodes['S1'].execute('sudo config interface ip add Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
print ("\n******************** Assign IP addresses on L0 *************************")
out = nodes['L0'].execute('sudo config interface ip add Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
print ("\n******************** Assign IP addresses on L1 *************************")
out = nodes['L1'].execute('sudo config interface ip add Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')

# Configure eBGP
print ("\n******************** Configure eBGP on S0 *************************")
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configure eBGP on S1 *************************")
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configure eBGP on L0 *************************")
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configure eBGP on L1 *************************")
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')
time.sleep(60)

> Configure the traffic generator interfaces.

In [ ]:
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig -a eth1')
out = nodes['trex'].execute('ifconfig -a eth2')

## Configure Static VXLANs over the 3-Stage Clos Network 

> In this section , the notebook demostrates how to configure static VXLAN tunnels between LEAF0 and LEAF1. The underlay consists of the 3-stage Clos network which we configured in the previous section. 


### Configure VLAN on Leaf Nodes

> Configure VLAN 10 on both LEAF routers to set up VXLAN Tunnels between the VLAN segments on the LEAFs. Then remove the IP addresses assigned to the LEAF interfaces connected to the traffic generator and assign the VLAN 10. Assign the IP address that we removed to the VLAN interface.

In [ ]:
out = nodes['L0'].execute('sudo config vlan add 10')
out = nodes['L0'].execute('sudo config vlan member add -u 10 Ethernet8')
out = nodes['L0'].execute('sudo config interface ip add Vlan10 10.0.5.1/24')
out = nodes['L0'].execute('show vlan brief')

out = nodes['L1'].execute('sudo config vlan add 10')
out = nodes['L1'].execute('sudo config vlan member add -u 10 Ethernet8')
out = nodes['L1'].execute('sudo config interface ip add Vlan10 10.0.6.1/24')
out = nodes['L1'].execute('show vlan brief')

### Set Up and Apply Static VXLAN Configurations

> Static VXLAN configurations are applied using JSON files. This step creates the files with the VXLAN configurations and loads them on LEAF0 and LEAF1. It configures VXLAN with a Virtual Network Identifier (VNI) of 1000 on LEAF0. The source IP address of the tunnel is the IP address of the Loopback 0 interface of LEAF0. This VXLAN maps VLAN 10 with a VNI of 1000.

In [ ]:
# This cell creates the json file
import json
  
# VXLAN Configurations
VXLAN_dictionary ={
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.10.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
         "Vlan10|10.0.5.1/24": {}
    }
}
    
json_object = json.dumps(VXLAN_dictionary, indent = 4)
# Writing to json file
with open("VXLAN.json", "w") as outfile:
    outfile.write(json_object)

# Copying json file to LEAF0    
rtr_ip = str(nodes['L0'].connections.cli.ip)
rtr_port = str(nodes['L0'].connections.cli.port)
src_file = "./VXLAN.json"
dst_on_rtr = "/home/cisco/VXLAN.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF0
out = nodes['L0'].execute('cat VXLAN.json')

# Apply and save the VXLAN configurations on LEAF0
out = nodes['L0'].execute('sudo config load VXLAN.json -y')
out = nodes['L0'].execute('sudo config save -y')

> Like LEAF0, on LEAF1 configure VXLAN with a VNI of 1000, which maps to VLAN 10. The source IP address of the VXLAN tunnel is the IP address of the Loopback0 interface of LEAF1.

In [ ]:
# This cell creates the json file  
# VXLAN Configurations
VXLAN_dictionary ={
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.11.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
         "Vlan10|10.0.6.1/24": {}
    }
}
json_object = json.dumps(VXLAN_dictionary, indent = 4)
  
# Writing to json file
with open("VXLAN2.json", "w") as outfile:
    outfile.write(json_object)

# Copying json file to LEAF1
rtr_ip = str(nodes['L1'].connections.cli.ip)
rtr_port = str(nodes['L1'].connections.cli.port)
src_file = "./VXLAN2.json"
dst_on_rtr = "/home/cisco/VXLAN2.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF1
out = nodes['L1'].execute('cat VXLAN2.json')

# Apply and save the VXLAN configurations on LEAF1
out = nodes['L1'].execute('sudo config load VXLAN2.json -y')
out = nodes['L1'].execute('sudo config save -y')

### Set Up and Load VNet Route Tables

> Since static VXLAN does not have a control plane to learn routes, set up the route tables manually by playing the below cell, SONiC uses the SWitch State Service (SWSS) docker container to maintain the database of routes. This cell sets up the JSON file for the Virtual Network (VNet) route table and copies it to the SWSS container using the ```docker cp``` command. The tables are then loaded in the SWSS container using the ```docker exec -i swss swssconfig``` command.

In [ ]:
# Setting up the VNET table with routes
VNET_ROUTE =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.0/24": {"endpoint": "10.10.11.200"},"OP": "SET"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.2/32": {"ifname": "Vlan10"},"OP": "SET"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE, indent = 4)
with open("VNET_ROUTE.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF0
rtr_ip = str(nodes['L0'].connections.cli.ip)
rtr_port = str(nodes['L0'].connections.cli.port)
src_file = "./VNET_ROUTE.json"
dst_on_rtr = "/home/cisco/vnt.route_snhop.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file with route table is copied on LEAF0
out = nodes['L0'].execute('cat vnt.route_snhop.json')

# Copy file with the route table over to swss container
out = nodes['L0'].execute('docker cp vnt.route_snhop.json swss://.')

# Load the routes in the swss container
out = nodes['L0'].execute('docker exec -i swss swssconfig vnt.route_snhop.json')

> Set up the route table on LEAF1

In [ ]:
# Setting up the VNET table with routes

VNET_ROUTE =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.0/24": {"endpoint": "10.10.10.200"},"OP": "SET"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.2/32": {"ifname": "Vlan10"},"OP": "SET"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE, indent = 4)
with open("VNET_ROUTE2.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF1
rtr_ip = str(nodes['L1'].connections.cli.ip)
rtr_port = str(nodes['L1'].connections.cli.port)   
src_file = "./VNET_ROUTE2.json"
dst_on_rtr = "/home/cisco/vnt.route_snhop.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF1
out = nodes['L1'].execute('cat vnt.route_snhop.json')

# Copy file with the route table over to swss container
out = nodes['L1'].execute('docker cp vnt.route_snhop.json swss://.')

# Load the routes in the swss container
out = nodes['L1'].execute('docker exec -i swss swssconfig vnt.route_snhop.json')

### Verify Static VXLAN

> Execute show commands on LEAF0 router to check the VXLAN configurations.

In [ ]:
out = nodes['L0'].execute('show vxlan interface')
out = nodes['L0'].execute('show vnet brief')
out = nodes['L0'].execute('show vnet name Vnet_1000')
out = nodes['L0'].execute('show ip route')

> Execute show commands on LEAF1 router to check the VXLAN configurations.

In [ ]:
out = nodes['L1'].execute('show vxlan interface')
out = nodes['L1'].execute('show vnet brief')
out = nodes['L1'].execute('show vnet name Vnet_1000')
out = nodes['L1'].execute('show ip route')

### Send Traffic across VXLAN Tunnel

> To simulate server-to-server traffic flow across VXLAN tunnel, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/vxlan-traffic.png" width="700"/></center>

> The function ```generate_bidir_traffic``` injects a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [ ]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

> Check the summary stats in the trex output above, for packet drops. And then verify the MAC addresses learnt on the leaf routers, L0 and L1.

In [ ]:
out = nodes['L0'].execute('show mac')
out = nodes['L1'].execute('show mac')

> You have now successfully brought up a VXLAN Tunnel over a 3-stage Clos network, sent traffic across the VXLAN. 

### Tested Static VXLAN Scale

> A scale of 32K remote Virtual Tunnel End Points (VTEP) with 128K unique overlay-prefix and VTEP Encap entries has been demonstrated on Cisco 8000 routers with SONiC. For more details of this demo by the Cisco team, refer https://blogs.cisco.com/sp/cisco-and-sonic

### Clean up Router Configurations

> Remove the router configurations so that you can play other notebooks

In [ ]:
# Cleaning up the VNET table
VNET_ROUTE_DEL =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.0/24": {"endpoint": "10.10.11.200"},"OP": "DEL"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.2/32": {"ifname": "Vlan10"},"OP": "DEL"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE_DEL, indent = 4)
with open("VNET_ROUTE_DEL.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF0
rtr_ip = str(nodes['L0'].connections.cli.ip)
rtr_port = str(nodes['L0'].connections.cli.port)
src_file = "./VNET_ROUTE_DEL.json"
dst_on_rtr = "/home/cisco/vnt.route_snhopdel.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file with route table is copied on LEAF0
out = nodes['L0'].execute('cat vnt.route_snhopdel.json')

# Copy file with the route table over to swss container
out = nodes['L0'].execute('docker cp vnt.route_snhopdel.json swss://.')

# Load the routes in the swss container
out = nodes['L0'].execute('docker exec -i swss swssconfig vnt.route_snhopdel.json')

In [ ]:
# Cleaning up the VNET table

VNET_ROUTE_DEL =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.0/24": {"endpoint": "10.10.10.200"},"OP": "DEL"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.2/32": {"ifname": "Vlan10"},"OP": "DEL"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE_DEL, indent = 4)
with open("VNET_ROUTE_DEL2.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF1
rtr_ip = str(nodes['L1'].connections.cli.ip)
rtr_port = str(nodes['L1'].connections.cli.port)   
src_file = "./VNET_ROUTE_DEL2.json"
dst_on_rtr = "/home/cisco/vnt.route_snhopdel.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF1
out = nodes['L1'].execute('cat vnt.route_snhopdel.json')

# Copy file with the route table over to swss container
out = nodes['L1'].execute('docker cp vnt.route_snhopdel.json swss://.')

# Load the routes in the swss container
out = nodes['L1'].execute('docker exec -i swss swssconfig vnt.route_snhopdel.json')


In [ ]:
# Remove the Router configs and release the interfaces used by trex
out = nodes['S0'].execute('sudo config interface ip remove Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Loopback0 10.10.10.100/32')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Loopback0 10.10.11.100/32')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Loopback0 10.10.10.200/32')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Loopback0 10.10.11.200/32')

# Unconfigure the VLANs
out = nodes['L0'].execute('sudo config interface ip remove Vlan10 10.0.5.1/24')
out = nodes['L0'].execute('sudo config vlan member del 10 Ethernet8')
out = nodes['L0'].execute('sudo config vlan del 10')

out = nodes['L1'].execute('sudo config interface ip remove Vlan10 10.0.6.1/24')
out = nodes['L1'].execute('sudo config vlan member del 10 Ethernet8')
out = nodes['L1'].execute('sudo config vlan del 10')
time.sleep(60)

for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')

In [50]:
!rm -rf *.json

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.